
**Генерация текстов на основе LSTM**


**Задание 1**

Загрузите текст из произведений Ницше ('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt').
Выведете следующее:

А) длину всего корпуса;

Б) количество предложений;

В) сколько всего символов используется?


In [ ]:
import io
import urllib.request
import nltk
import numpy as np
import random
import sys
import sqlite3
import pandas as pd
import requests
import re
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback
from keras.layers import Activation

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
url = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt'
filename = 'nietzsche.txt'
urllib.request.urlretrieve(url, filename=filename)

with io.open(filename, encoding='utf-8') as f:
    text = f.read().lower()

chars = list(set(text))

sentence = nltk.sent_tokenize(text)

print('А) Длина всего корпуса:', len(text))
print('Б) Количество предложений:', len(sentence))
print('В) Всего символов:', len(chars))

**Задание 2**

Сократите текст наполовину избыточными последовательностями символов maxlen

In [ ]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.uint8)
y = np.zeros((len(sentences), len(chars)), dtype=np.uint8)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

**Задание 3**

Создайте модель LSTM для генерации текста

А) Напишите вспомогательную функцию для выборки индекса из массива вероятностей

Б) Напишите функцию, которая будет вызываться в конце каждой эпохи и печатать сгенерированный текст

В) Запустите модель на обучение
Имейте ввиду, что требуется не менее 20 эпох, прежде чем сгенерированный текст начнет звучать связно. Рекомендуется запускать этот скрипт на графическом процессоре, так как рекуррентные сети требуют довольно больших вычислительных затрат.

Г) Проверьте работу модели в онлайн режиме.


In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

A)

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Б)

In [ ]:
def on_epoch_end(epoch, _):
    print()


    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('РАЗНООБРАЗИЕ:', diversity)

        generated = ''
        sentence = list(text[start_index: start_index + maxlen])
        generated += ''.join(sentence)
        print('НАШ ЗАПРОС НЕЙРОННОЙ СЕТИ: "' + ''.join(sentence) + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                try:
                  x_pred[0, t, char_indices[char]] = 1
                except:
                  continue

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + [next_char]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

B)

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
callbacks=[print_callback])

Epoch 1/20
1564/1565 [============================>.] - ETA: 0s - loss: 2.0334
EPOCH: 0
РАЗНООБРАЗИЕ: 0.2
НАШ ЗАПРОС НЕЙРОННОЙ СЕТИ: " of moral valuations--they are still
ult"
 of moral valuations--they are still
ulternated the presente of the sole the presented to the the most the presintion of the which the possible the socrite of the presented of the out the man the presented and the presented and the presented to the incention of the the the precise and the presented the the consected and the most the presintion of the respeal person in the the the and the endimented and all the presented and the most all
РАЗНООБРАЗИЕ: 0.5
НАШ ЗАПРОС НЕЙРОННОЙ СЕТИ: " of moral valuations--they are still
ult"
 of moral valuations--they are still
ultay the invount of the history of chiloged has it still every precise and in the may the presence of may the poction of ble one which is the soul the soul to be a sere
one the remall to moral the right and all the himself and his of the many when the very 

Г)

In [ ]:
def generate_text(model, start_string, generation_length=400):

    generated_text = start_string
    for i in range(generation_length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(start_string):
            x_pred[0, t, char_indices[char]] = 1

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 1.0)
        next_char = indices_char[next_index]

        start_string = start_string[1:] + next_char
        generated_text += next_char

    return generated_text

start_string = input("Введите начальную строку: ")
generated_text = generate_text(model, start_string)
print("Сгенерированный текст:\n", generated_text)

Введите начальную строку: tude, and what is most poisonous in it
Сгенерированный текст:
 tude, and what is most poisonous in itya-sook-yf
eoaw"i.ayoe.ssi-ae"met roafs.ieslsi-ye.uti-pwov"ia.yu-pffu uofoernpiwoyltu-hteissi;h-mtmrc
elue
aci-gya
uh,yiws",heh-mtfmdi.esasi.btco"e,-mhe,cihpenenh iuhrnole,eonn-uwgvi,yyi.htu"lghi maffhclom orbs,mo rabi ufbmrwa"ud"w
pmwiiw.u ay,oouoi wpleoi uci fft tow
afgtdi iyi m rtonnnsfbiaeeyosynt ipayf,-pwk tith-vayvoemf-nmyn:yi" mawurele hhyr e-nuwmm-.-oeni"ptbnui"p"lbni"-eoop"u
oartisi-iaskt


**Задание 4**

Создайте самостоятельно генерацию текста для РУССКОЯЗЫЧНОГО НАБОРА глав Wikibooks.
Полный текст Wikibooks содержит более 270000 глав на 12 языках https://www.kaggle.com/datasets/dhruvildave/wikibooks-dataset/data

In [ ]:
connect = sqlite3.connect('/content/drive/MyDrive/wikibooks_data_ru.sqlite')
cursor = connect.cursor()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

tables = cursor.fetchall()
tables

[('wikibooks_table',)]

In [ ]:
cursor.execute("SELECT body_text FROM wikibooks_table")
text = cursor.fetchall()

In [ ]:
text = ' '.join([t[0].lower() for t in text])

In [ ]:
text = re.sub(r'[^а-яА-ЯёЁ\s.]', '', text)
text = re.sub(r'\s+', ' ', text)
text = text.strip()
print(text[:200])

In [ ]:
text = text[:200000]

In [ ]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
maxlen = 40
step = 5

sentences_2 = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences_2.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences_2))

nb sequences: 39992


In [ ]:
x = np.zeros((len(sentences_2), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences_2), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences_2):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
with tf.device('/GPU:0'):
    model = Sequential()
    model.add(LSTM(256, input_shape=(maxlen, len(chars))))
    model.add(Dense(len(chars)))
    model.add(Activation('softmax'))

    optimizer = RMSprop(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
with tf.device('/GPU:0'):

    model.fit(x, y,
              batch_size=256,
              epochs=20,
    callbacks=[print_callback])

Epoch 1/20
155/157 [============================>.] - ETA: 0s - loss: 3.0909
РАЗНООБРАЗИЕ: 0.2
НАШ ЗАПРОС НЕЙРОННОЙ СЕТИ: "цию слг страховкапроцедуры по пилотупило"
цию слг страховкапроцедуры по пилотупило  аооераа  е ааа а ееа не оте ае ооаа ио еоое раен са оаа ое алоа е аа а оота а еее а ооае ее ое о аае отоаеот оето ае о аоераа тае е аае  оаеи еаа а оее  оае орае е лее ае аоне ене еа а е аое ао нореие ооа оа ете е орое е еаа раа а оава аоо оо аое олее те ое  а есео то аео ее еокаа еена  аа аое е онае ао оото оо ооооо де ое оа а ае а ооероре ате о оеале аое о аее еаа  аата ао аае и аате оаа а аот
РАЗНООБРАЗИЕ: 0.5
НАШ ЗАПРОС НЕЙРОННОЙ СЕТИ: "цию слг страховкапроцедуры по пилотупило"
цию слг страховкапроцедуры по пилотупилоеенааенеий иеепи иеонее. равиов раеанеен оовтерекн и прек ае ми тоы бодооминттеве снтс раотсрок аная еора нраао ево  р ора реоптниеа ве е раееи дое улщаоте ла ердааариаветторе ыеекааемам оеарлоооа наваооа ррсел виеинн оаелв енр неиоо зднрлеиал  ое еиетнаок еопееето 

In [ ]:
def generate_text(model, start_string, generation_length=400):

    generated_text = start_string
    for i in range(generation_length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(start_string):
            x_pred[0, t, char_indices[char]] = 1

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 1.0)
        next_char = indices_char[next_index]

        start_string = start_string[1:] + next_char
        generated_text += next_char

    return generated_text

start_string = input("Введите начальную строку: ")
generated_text = generate_text(model, start_string)
print("Сгенерированный текст:\n", generated_text)

Введите начальную строку: документов содержится
Сгенерированный текст:
 документов содержится  нии  зеетвоёмленнккоюи ео о.ннсо ро   ии  р  в  ан.к мкмко.  мкмь.рйвк..   иии воринсе к  тнав   аонновнкитф ое    щмс.дм н  аие   й  и      т     няя ии и бклпи л.лкок. и  л киен    в саее г.иуа тккни...      й ил.освккоиы ие т   им.н нн..е  ме   г нлхее  г кт  заи алвицон   ккн   ат   яиллипнкитвквклкек кооусхнонн н ои  иовсдлдлнннввснннвнпнн орекы лвккц кее м   и   н ннмиело мнс  ни   нпе нлн
